In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Lecture 24 #

## Reviewing Setup from Last Lecture

Recall, in our example from last class we actually had access to the population data.  
This is not normally the case.  The methods we are about to study **only make sense when you don't have access to the population.** 

In [ ]:
sf = pd.read_csv('san_francisco_2019.csv')
min_salary = 15 * 20 * 52
sf = sf[sf["Salary"]>=min_salary]
sf


In [ ]:
sf_bins = np.arange(0, 726000, 50000)
sf.hist('Total Compensation', bins=sf_bins)

### Defining the Statistic

Here we are interested in the median (50% percentile) of the total compensation. 

In [ ]:
# Parameter: Median total compensation in the population
def median_comp(t):
    return t['Total Compensation'].median()

### The Population Parameter

We have access to the population so we can compute the **parameter** but in practice we typically won't have access to the population and instead we will have to *estimate* the parameter from a **sample**.

In [ ]:
pop_median = median_comp(sf)
print("Parameter Value:", pop_median)

### The Sample

In practice, the data we would get from most studies would be a **sample** of the entire population.  This is because collecting a full census is challenging and expensive.

Here we load the (fictional) sample that was taken by Prof. Oscamou

<details><summary>Unimportant and Not True Details</summary>

She collected this sample by selecting 400 names at random from the HR database and then directly asking them how much they make. It took all weekend and he had to give a chocolate to everyone who participated. 
    
    
<details><summary>The Real Truth</summary>
        
Actually, everything above is a complete fabrication.  Prof. Oscamou simply wrote: 
        
```python
sf.sample(400, replace=False).to_csv("sf_sample.csv")
```
        
</details>
    
</details>

In [ ]:
sample_sf = pd.read_csv("sf_sample.csv")
sample_sf.head(5)

In [ ]:
#Make sure our sample has the correct number of rows:

sample_sf.shape[0]

### Bootstrap Sampling

We introduced the following function that computes bootstrap samples of the statistic.
Note we recommend you use **10,000** repetitions when computing bootstrapped distributions.

In [ ]:
def bootstrapper(sample, statistic, num_repetitions=10000):
    """
    Returns the statistic computed on a num_repetitions  
    bootstrap samples from sample.
    """
    bstrap_stats = np.array([])
    for i in np.arange(num_repetitions):
        # Step 1: Sample the Sample
        bootstrap_sample = sample.sample(frac=1, replace=True)
        # Step 2: compute statistics on the sample of the sample
        bootstrap_stat = statistic(bootstrap_sample)
        # Accumulate the statistics
        bstrap_stats = np.append(bstrap_stats, bootstrap_stat)

    return bstrap_stats    

In [ ]:
bootstrap_medians = bootstrapper(sample_sf, median_comp, 10000)

### Examining the Bootstrapped Distribution of the Statistic

When using the boostrap it is important to always examine the distribution of the statistic.  In general when using bootstrapping we are looking for a distribution that is **roughly symmetric and bell-shaped**.

In [ ]:
plt.hist(bootstrap_medians, density=True, ec='white');

parameter_green = '#32CD32'
plt.ylim(-0.000005, 0.00014)
plt.scatter(pop_median, 0, color=parameter_green, s=40, zorder=2)
plt.title('Bootstrap Medians and the Parameter (Green Dot)');

### Computing the Confidence Interval

We compute the confidence interval for our desired **confidence level** using the following code:

In [ ]:
def compute_ci(bs_samples, confidence_level):
    """
    Returns the confidence interval for the provided bootstrap samples
    and desired confidence level.
    """
    tail_size = (100 - confidence_level)/2
    lower = np.percentile(bs_samples, tail_size,)
    upper = np.percentile(bs_samples, 100 - tail_size, )
    return [lower, upper]

In [ ]:
ci = compute_ci(bootstrap_medians, 95)
ci

Visualizing the CI

In [ ]:
plt.hist(bootstrap_medians, density=True, ec='white');
# cool python trick to deconstruct the array!
[left, right] = ci
# Plotting parameters; you can ignore this code
plt.ylim(-0.000005, 0.00014)
plt.plot([left, right], [0, 0], color='yellow', lw=5, zorder=1)
plt.scatter(pop_median, 0, color=parameter_green, s=40, zorder=2);

<br><br><br><br>

---

Return to Slides

---

<br><br><br><br>


---

## Simulating the Simulation!

Let's look at what happens if we use the above code repeatedly with separate original samples from the population? How accurate are our 95% Bootstrap Estimates of the Parameter? 

In [ ]:
intervals = pd.DataFrame(columns = ['lower', 'upper', 'median', 'good', 'sample size' ])

sample_sizes = [2,8,16,50, 100]
for sample_size in sample_sizes:
    for trial in np.arange(20): # Run 20 trials of each configuration
        # Pay for one new random sample from the population
        og_sample = sf.sample(sample_size, replace=False)
        
        # Compute the statistic on the sample
        sample_median = median_comp(og_sample)
        
        # Generate the medians of 10000 bootstrap samples
        bootstrap_medians = bootstrapper(og_sample, median_comp, 1000)
        
        # Construct the confidence interval
        [ci_lower, ci_upper] = compute_ci(bootstrap_medians, 95)
        
        # Determine if the confidence interval is good
        is_good = ci_lower <= pop_median <= ci_upper
        
        # Add a row with this data to the DataFrame
        
        intervals.loc[len(intervals.index)] = [ci_lower, ci_upper, sample_median, is_good, str(sample_size)]
    

        

# Add an experiment number
intervals["Experiment"]= np.arange(intervals.shape[0])

In [ ]:
intervals

Here I render a plot of all the confidence intervals with the true value depicted as a solid vertical line.  

In [ ]:
import plotly.express as px

# Plotly will draw error bars which are the distance in each direction
# from the median
intervals["error_left"] = intervals["median"]-intervals["lower"]
intervals["error_right"] = intervals["upper"]-intervals["median"]


# Making the plot
fig = px.scatter(
    x=intervals["median"], # X location of my interval center dot
    y=intervals["Experiment"], # Y location of my interval center dot 
    color=intervals["sample size"], # The color to use.
    symbol=intervals["good"], # The column to use for the symbol
    symbol_map={True: "circle", False: "circle-open"},
    error_x=intervals["error_right"], # upper error bar size
    error_x_minus=intervals["error_left"], # lower error bar size
    height=800)
fig.add_vline(pop_median)

<br><br><br><br>

---

Return to Slides

---

<br><br><br><br>


## Confidence Interval for Unknown Population Mean

Now let's look at a more appropriate use of the bootstrap (when we don't have the population).  The baby table from prior lecture had a random sample of moms.

In [ ]:
# Random sample of mother-newborn pairs
births = pd.read_csv('baby.csv')
births

What is the average age of moms who are having children in the entire population?

**Parameter:** Average age of moms when they give birth to first child.

**Statistic:** Average age of moms in our sample.

In [ ]:
# Average age of mothers in the sample
births['Maternal Age'].mean()

Remember there was a distribution of ages.

In [ ]:
births.hist('Maternal Age')
plt.title("Distribution of Age in our SAMPLE")


We could have also returned the median or even a range of ages as our statistic:

In [ ]:
births['Maternal Age'].median()

Or an interval of 95% of the ages:

In [ ]:
[np.percentile(births['Maternal Age'],2.5),
 np.percentile(births['Maternal Age'],97.5)]

Is this a confidence interval?
<br><br><br><br>

### Compute the Sample Statistic
Since we are interested in estimating the average age of mothers in the population we will use the average age statistic:

In [ ]:
def avg_maternal_age(sample):
    return sample["Maternal Age"].mean()

In [ ]:
sample_statistic = avg_maternal_age(births)
sample_statistic

### Use the Bootstrap to Estimate the CI 

The interval of estimates is the "middle 95%" of the bootstrap estimates.

This is called a *95% confidence interval* for the **mean age in the population**.


In [ ]:
bootstrap_means = bootstrapper(births, avg_maternal_age, 10000)
avg_maternal_age_ci = compute_ci(bootstrap_means, confidence_level=95)
avg_maternal_age_ci

In [ ]:
plt.hist(bootstrap_means, density=True, ec='white');


[left, right] = avg_maternal_age_ci
plt.plot([left, right], [0, 0], color='yellow', lw=8);

In [ ]:
# Now we can visualize this interval back on our sample
# Notice tiny yellow bar at the bottom:
births.hist('Maternal Age')
plt.title("Distribution of Age in our SAMPLE")

plt.plot([left, right], [0, 0], color='yellow', lw=10);

<br><br><br><br>

---

Return to Slides

---

<br><br><br><br>


$\sqrt{p(1-p)}$

## Using the Confidence Interval for Testing Hypotheses

**Null:** The average age of mothers in the population is 25 years; the random sample average is different due to chance.

**Alternative:** The average age of the mothers in the population is **not** 25 years.

Suppose you use the 5% cutoff for the p-value.

Based on the confidence interval, which hypothesis would you pick?

In [ ]:
bootstrap_means = bootstrapper(births, avg_maternal_age, 10000)
avg_maternal_age_ci = compute_ci(bootstrap_means, confidence_level=95)
avg_maternal_age_ci

Suppose you use the 1% cutoff for the p-value.


In [ ]:
bootstrap_means = bootstrapper(births, avg_maternal_age, 10000)
avg_maternal_age_ci = compute_ci(bootstrap_means, confidence_level=99)
avg_maternal_age_ci

check it: https://www.cdc.gov/nchs/nsfg/key_statistics/b.htm

<br><br><br><br>

---

Return to Slides

---

<br><br><br><br>


## Using CLT to Calculate the CI for Age:



In [ ]:
sample_mean=births['Maternal Age'].mean()
sample_mean

In [ ]:
standard_error = births['Maternal Age'].std()/np.sqrt(len(births["Maternal Age"]))
standard_error

In [ ]:
#95% CI
L = sample_mean-2*standard_error
U = sample_mean+2*standard_error

[L,U]

In [ ]:
#Compare to bootstrapped example:
avg_maternal_age_ci